In [1]:
from matplotlib.colors import LogNorm
import numpy as np
import pandas as pd
import os
import glob
from datetime import datetime
from datetime import timedelta
from matplotlib import pyplot as plt
import matplotlib.dates as md
import matplotlib.colors as mcolors
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import warnings
from mpl_toolkits.basemap import Basemap
from scipy.interpolate import interp2d
warnings.filterwarnings('ignore')
#import datetime
import scipy.ndimage as ndimage
from matplotlib import cm
import geopy.distance
#import matplotlib as mpl

from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame
import leafmap
import plotly.express as px

from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm
#import pysplit

In [17]:
def read_traj(path):
    with open(path) as f:
        content = f.readlines()
    content = [x.strip() for x in content]
    skiprows = int(content[0].split(' ')[0])+4
    traj = pd.read_csv(path, skiprows=skiprows, header=None, delim_whitespace=True)
    traj = traj.iloc[:, 2:]
    traj.columns = ['Year', 'Month', 'Day', 'Hour', 'Min', '',
                    'Timestep', 'Latitude', 'Longitude', 'Altitude_AGL_m',
                    'Pressure_mb', 'tpot', 'tamb',
                    'rain', 'mixd', 'relh', 'sphu',
                    'mixr', 'dswf', 'terr']
    traj['Launch_UTC'] = pd.to_datetime(f'{traj.Year.iloc[0].astype(str)}'
                                        f'{traj.Month.iloc[0].astype(str).zfill(2)}'
                                        f'{traj.Day.iloc[0].astype(str).zfill(2)}'
                                        f'{traj.Hour.iloc[0].astype(str).zfill(2)}'
                                        f'{traj.Min.iloc[0].astype(str).zfill(2)}',
                                        format='%y%m%d%H%M')
    traj.drop(columns=['Year', 'Month', 'Day', 'Hour', 'Min', ''], inplace=True)
    traj['Launch_Altitude_AGL_m'] = traj.Altitude_AGL_m[0]
    return traj

In [19]:
# Get a list of all file paths matching the glob pattern
file_paths = glob.glob('/Users/taiwoajayi/Documents/HYSPLIT_Bermuda/ACTIVATE_B_P_GFS_72h'+'/*ACTIVATE*20220602*')  # Replace 'path_to_files' with the actual path and glob pattern
file_paths

['/Users/taiwoajayi/Documents/HYSPLIT_Bermuda/ACTIVATE_B_P_GFS_72h\\ACTIVATE_B_P_GFS_72h_202206021123',
 '/Users/taiwoajayi/Documents/HYSPLIT_Bermuda/ACTIVATE_B_P_GFS_72h\\ACTIVATE_B_P_GFS_72h_202206021124',
 '/Users/taiwoajayi/Documents/HYSPLIT_Bermuda/ACTIVATE_B_P_GFS_72h\\ACTIVATE_B_P_GFS_72h_202206021125',
 '/Users/taiwoajayi/Documents/HYSPLIT_Bermuda/ACTIVATE_B_P_GFS_72h\\ACTIVATE_B_P_GFS_72h_202206021126',
 '/Users/taiwoajayi/Documents/HYSPLIT_Bermuda/ACTIVATE_B_P_GFS_72h\\ACTIVATE_B_P_GFS_72h_202206021127',
 '/Users/taiwoajayi/Documents/HYSPLIT_Bermuda/ACTIVATE_B_P_GFS_72h\\ACTIVATE_B_P_GFS_72h_202206021128',
 '/Users/taiwoajayi/Documents/HYSPLIT_Bermuda/ACTIVATE_B_P_GFS_72h\\ACTIVATE_B_P_GFS_72h_202206021129',
 '/Users/taiwoajayi/Documents/HYSPLIT_Bermuda/ACTIVATE_B_P_GFS_72h\\ACTIVATE_B_P_GFS_72h_202206021130',
 '/Users/taiwoajayi/Documents/HYSPLIT_Bermuda/ACTIVATE_B_P_GFS_72h\\ACTIVATE_B_P_GFS_72h_202206021131',
 '/Users/taiwoajayi/Documents/HYSPLIT_Bermuda/ACTIVATE_B_P_GFS_7

In [27]:
# Initialize an empty list to store the individual DataFrames
dfs = []

# Iterate over each file path
for file_path in file_paths:
    # Read each file as a DataFrame using the read_traj() function
    df = read_traj(file_path)
    # Append the DataFrame to the list
    dfs.append(df)

# Concatenate all DataFrames in the list into one DataFrame
merged_df = pd.concat(dfs, ignore_index=True)
merged_df

# Save the merged DataFrame to a file named "merge.csv"
#merged_df.to_csv('merge.csv', index=False)


,Timestep,Latitude,Longitude,Altitude_AGL_m,Pressure_mb,tpot,tamb,rain,mixd,relh,sphu,mixr,dswf,terr,Launch_UTC,Launch_Altitude_AGL_m
0,0.0,32.437,-64.956,1531.6,848.7,300.8,287.0,0.0,646.9,71.6,8.3,8.4,0.0,79.4,2022-06-02 11:23:00,1531.6
1,-0.4,32.442,-65.065,1501.1,851.7,300.7,287.2,0.0,643.6,72.6,8.4,8.6,0.0,66.7,2022-06-02 11:23:00,1531.6
2,-1.4,32.440,-65.350,1404.4,861.6,300.3,287.8,0.1,702.5,70.7,8.5,8.6,0.0,33.2,2022-06-02 11:23:00,1531.6
3,-2.4,32.426,-65.638,1305.4,869.9,299.9,288.2,0.2,739.0,72.8,8.9,9.0,0.0,0.0,2022-06-02 11:23:00,1531.6
4,-3.4,32.403,-65.924,1203.6,883.7,299.3,288.9,0.0,748.6,74.6,9.3,9.5,0.0,0.0,2022-06-02 11:23:00,1531.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1820,-67.8,31.465,-70.905,53.7,1014.7,296.2,297.4,0.0,425.2,76.3,14.1,14.5,0.0,669.7,2022-06-02 11:47:00,123.8
1821,-68.8,31.415,-70.900,59.5,1013.9,296.1,297.3,0.0,402.6,77.8,14.3,14.6,0.0,633.8,2022-06-02 11:47:00,123.8
1822,-69.8,31.360,-70.894,62.2,1013.5,296.0,297.2,0.3,390.1,78.6,14.4,14.7,0.0,448.3,2022-06-02 11:47:00,123.8
1823,-70.8,31.297,-70.884,63.6,1013.4,295.9,297.1,0.3,358.4,79.6,14.4,14.8,0.0,257.4,2022-06-02 11:47:00,123.8


In [25]:
# Reorder the columns
new_order = ['Timestep', 'Launch_UTC','Latitude', 'Longitude', 'Launch_Altitude_AGL_m', 'Altitude_AGL_m', 'Pressure_mb','tpot','tamb', 'rain', 'mixd', 'relh', 'sphu' 'mixr', 'dswf', 'terr']
merged_df = merged_df.reindex(columns=new_order)
print(merged_df)

      Timestep          Launch_UTC  Latitude  Longitude   
0          0.0 2022-06-02 11:23:00    32.437    -64.956  \
1         -0.4 2022-06-02 11:23:00    32.442    -65.065   
2         -1.4 2022-06-02 11:23:00    32.440    -65.350   
3         -2.4 2022-06-02 11:23:00    32.426    -65.638   
4         -3.4 2022-06-02 11:23:00    32.403    -65.924   
...        ...                 ...       ...        ...   
1820     -67.8 2022-06-02 11:47:00    31.465    -70.905   
1821     -68.8 2022-06-02 11:47:00    31.415    -70.900   
1822     -69.8 2022-06-02 11:47:00    31.360    -70.894   
1823     -70.8 2022-06-02 11:47:00    31.297    -70.884   
1824     -71.8 2022-06-02 11:47:00    31.229    -70.872   

      Launch_Altitude_AGL_m  Altitude_AGL_m  Pressure_mb   tpot   tamb  rain   
0                    1531.6          1531.6        848.7  300.8  287.0   0.0  \
1                    1531.6          1501.1        851.7  300.7  287.2   0.0   
2                    1531.6          1404.4        

In [26]:
start_index = np.where(merged_df["Launch_UTC"]==pd.to_datetime("2022-06-02 13:48:00"))
stop_index = np.where(merged_df["Launch_UTC"]==pd.to_datetime("2022-06-02 14:10:00"))   
merged_df2_subset = merged_df[start_index[0][0]:stop_index[0][0]]
merged_df2_subset

IndexError: index 0 is out of bounds for axis 0 with size 0